In [28]:
# Read the general DTS and save the taxonomy schema to PostgreSQL
from hibachiPostgreSQL import ReadAllGeneralDTSPathesInPostgre # For getting file general DTS pathes
from hibachixbrl import HibachiXmlElement, HibachiETtree
import pandas as pd
import sys
import os

In [29]:
# Get all the general DTS file pathes saved in PostgreSQL 
df_DTSPathes = ReadAllGeneralDTSPathesInPostgre.generalDTSPathes()

Connecting to the PostgreSQL database...
Connection is closed.


In [35]:
def readXSDinGeneralDTS(df_generalDTS):
    ''' Get all the TaxonomySchema, Linkbase, FootnoteLink, Roletype classes in .xsd file.'''

    for CurrentPathName in df_generalDTS['filepath']:
        #Get the extention of the current file
        CurrentFileName = os.path.split(CurrentPathName)[1]
        CurrentFileExtension = os.path.splitext(CurrentPathName)[1] 
        if CurrentFileExtension == '.xsd':
            
            # Dataframes for TaxonomySchema, Linkbase, RoleTypes
            df_TaxonomySchema = pd.DataFrame(columns=['filepath','TaxonomySchemaObj'])
            df_Linkbases      = pd.DataFrame(columns=['filepath','linkbaseObj'])
            df_RoleTypes      = pd.DataFrame(columns=['filepath','roletypeObj'])    
        
            try:
                root_ElementTree = HibachiETtree.etree_parse_remove_NS(CurrentPathName)
                if root_ElementTree != None:
                    
                    # print the current file name
                    # print('Reading', CurrentFileName ,' ...')
                    # List of tags for taxonomy schema, linkbase and roleType/Refs
                    schema_tags = ['import','include','schemaRef','linkbaseRef','roleRef', 'arcroleRef']
                    linkbase_tags = ['linkbase','schema']
                    search_tags = schema_tags + linkbase_tags
                    
                    # Search elements with the tags listed above and corresponding create instances
                    for empty_Var, search_tagName in enumerate(search_tags):
                        for schema_ElementTree in root_ElementTree.iter(search_tagName):
                            # Taxonomy Schema
                            if search_tagName in schema_tags:
#                                 print(CurrentPathName,schema_ElementTree)
                                df_TaxonomySchema = df_TaxonomySchema.append(\
                                    {'filepath':CurrentPathName,\
                                     'TaxonomySchemaObj':HibachiXmlElement.TaxonomySchema(CurrentFileName, schema_ElementTree)}\
                                    ,ignore_index=True
                                )
                                
                            # 5.1.2 <linkbase>
                            elif search_tagName == 'linkbase':
                                df_Linkbases = df_Linkbases.append(\
                                    {'filepath':CurrentPathName,\
                                     'linkbaseObj':HibachiXmlElement.Linkbase(CurrentFileName, schema_ElementTree)}\
                                    ,ignore_index=True
                                )
                                
                            # 5.1.3 <roleType>, 5.1.4 <arcroleType>
#                             5.1.3 Defining custom role types - the <roleType> element
#                             The <roleType> element MUST be located among the set of nodes identified by the [XPath 1.0] path "//xsd:schema/xsd:annotation/xsd:appinfo/*".
#                             The value of the @roleURI attribute identifies the @xlink:role attribute value that is being defined.
#                             5.1.4 Defining custom arc role types - the arcroleType element
#                             The <arcroleType> element MUST be among the set of nodes identified by the [XPath 1.0] path "//xsd:schema/xsd:annotation/xsd:appinfo/*".
#                             The value of the @arcroleURI identifies the @xlink:arcrole attribute value that is being defined.

                            elif search_tagName == 'schema':
                                for annotation_ElementTree in schema_ElementTree.findall('annotation'):
                                    for appinfo_ElementTree in annotation_ElementTree.findall('appinfo'):
                                        
                                        roleType_Tags = ['roleType','arcroleType']
                                        for empty_Var, temp_tagName in enumerate(roleType_Tags):
                                            for temp_roleType_ElementTree in appinfo_ElementTree.findall(temp_tagName): #
                                                print(CurrentPathName,temp_roleType_ElementTree)
                                                df_RoleTypes = df_RoleTypes.append(\
                                                    {'filepath':CurrentPathName,\
                                                     'roletypeObj':HibachiXmlElement.Roletype(CurrentFileName, temp_roleType_ElementTree)}\
                                                    ,ignore_index=True
                                                )
                                                
                                        roleRef_Tags = ['roleRef', 'arcroleRef']
                                        for empty_Var, temp_tagName in enumerate(roleRef_Tags):
                                            for temp_roleRef_ElementTree in appinfo_ElementTree.findall(temp_tagName):
                                                df_TaxonomySchema = df_TaxonomySchema.append(\
                                                    {'filepath':CurrentPathName,\
                                                     'TaxonomySchemaObj':HibachiXmlElement.TaxonomySchema(CurrentFileName, schema_ElementTree)}\
                                                    ,ignore_index=True
                                                )
                            else:
                                print('Tag is:',search_tagName, ' . Ignored.')
                else:
                    print('No root element')

            except:
                print ('Error in reading:', CurrentPathName, sys.exc_info()[0], sys.exc_info()[1])
    return df_TaxonomySchema, df_Linkbases, df_RoleTypes

In [ ]:
# Run the function
df_Taxonomy, df_Linkbase, df_RoleType = readXSDinGeneralDTS(df_generalDTS=df_DTSPathes)

EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127B2C860>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127C876D0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127C87B30>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127C87DB0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127C84130>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127C84450>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127C84770>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127C84A90>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127C84DB0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_r

EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x000002312713A9A0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x000002312713ACC0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127133040>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127133360>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127133680>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x00000231271339A0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023127133CC0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x000002312712B040>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x000002312712B360>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_r

EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023126F76540>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023126F768B0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023126F76BD0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023126F76EF0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023126F98270>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023126F98590>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023126F988B0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023126F98BD0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_rt_2013-08-31.xsd <Element 'roleType' at 0x0000023126F98EF0>
EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_r

EdinetGeneraltaxonomy\jpcrp\2015-03-31/jpcrp_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023125315EA0>
EdinetGeneraltaxonomy\jpcrp\2015-03-31/jpcrp_rt_2015-03-31.xsd <Element 'roleType' at 0x000002312530F220>
EdinetGeneraltaxonomy\jpcrp\2015-03-31/jpcrp_rt_2015-03-31.xsd <Element 'roleType' at 0x000002312530F540>
EdinetGeneraltaxonomy\jpcrp\2015-03-31/jpcrp_rt_2015-03-31.xsd <Element 'roleType' at 0x000002312530F860>
EdinetGeneraltaxonomy\jpcrp\2015-03-31/jpcrp_rt_2015-03-31.xsd <Element 'roleType' at 0x000002312530FB80>
EdinetGeneraltaxonomy\jpcrp\2015-03-31/jpcrp_rt_2015-03-31.xsd <Element 'roleType' at 0x000002312530FEA0>
EdinetGeneraltaxonomy\jpcrp\2015-03-31/jpcrp_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231252ED220>
EdinetGeneraltaxonomy\jpcrp\2015-03-31/jpcrp_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231252ED540>
EdinetGeneraltaxonomy\jpcrp\2015-03-31/jpcrp_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231252ED860>
EdinetGeneraltaxonomy\jpcrp\2015-03-31/jpcrp_r

EdinetGeneraltaxonomy\jpcrp\2016-02-29/jpcrp_rt_2016-02-29.xsd <Element 'roleType' at 0x00000231255497C0>
EdinetGeneraltaxonomy\jpcrp\2016-02-29/jpcrp_rt_2016-02-29.xsd <Element 'roleType' at 0x0000023127CC3D10>
EdinetGeneraltaxonomy\jpcrp\2016-02-29/jpcrp_rt_2016-02-29.xsd <Element 'roleType' at 0x0000023127CD2090>
EdinetGeneraltaxonomy\jpcrp\2016-02-29/jpcrp_rt_2016-02-29.xsd <Element 'roleType' at 0x0000023127CD23B0>
EdinetGeneraltaxonomy\jpcrp\2016-02-29/jpcrp_rt_2016-02-29.xsd <Element 'roleType' at 0x0000023127CD26D0>
EdinetGeneraltaxonomy\jpcrp\2016-02-29/jpcrp_rt_2016-02-29.xsd <Element 'roleType' at 0x0000023127CD29F0>
EdinetGeneraltaxonomy\jpcrp\2016-02-29/jpcrp_rt_2016-02-29.xsd <Element 'roleType' at 0x0000023127CD2D10>
EdinetGeneraltaxonomy\jpcrp\2016-02-29/jpcrp_rt_2016-02-29.xsd <Element 'roleType' at 0x0000023124F90090>
EdinetGeneraltaxonomy\jpcrp\2016-02-29/jpcrp_rt_2016-02-29.xsd <Element 'roleType' at 0x0000023124F903B0>
EdinetGeneraltaxonomy\jpcrp\2016-02-29/jpcrp_r

...

EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023127FBB090>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023127FBBE00>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231270F3A40>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231270F3CC0>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231270F39F0>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231270F3130>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023125459450>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x000002312710ED60>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x000002312710E630>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_r

EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124FD96D0>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124FD9810>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124FD9950>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124FD9A90>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124FD9BD0>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124FD9D10>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124FD9E00>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124FD9F40>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124FCD090>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_r

EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124E648B0>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124E649A0>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124E64A90>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124E64B80>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124E64C70>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124E64D60>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124E64E50>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124E64F40>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023124E77090>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_r

EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231278A3040>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231278A3180>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231278A32C0>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231278A3400>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231278A3540>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231278A3630>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231278A3770>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231278A3860>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x00000231278A3950>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_r

EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023127E02EF0>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023127E0BF90>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023127E0BEA0>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023127E0BD60>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023127E0BC70>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023127E0BB30>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023127E0BA40>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023127E0B950>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_rt_2015-03-31.xsd <Element 'roleType' at 0x0000023127E0B860>
EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_r

In [32]:
df_Taxonomy

,filepath,TaxonomySchemaObj


In [11]:
# Unfold the Taxonomy Schema instace to Series and put it into dataframe
# Concate the unfolded dataframe to the original dataframe
#df_Taxonomy = pd.concat([df_Taxonomy.drop(['TaxonomySchemaObj'], axis=1), df_TaxonomySchema], axis=1)
df_TaxonomyForSQL = df_Taxonomy['TaxonomySchemaObj'].apply(lambda x: pd.Series(x.__dict__))
df_RoleTypeForSQL = df_RoleType['roletypeObj'].apply(lambda x: pd.Series(x.__dict__))

In [119]:
# Save the dataframe containing general DTS to PostgreSQL
# Connect to PostgreSQL
from sqlalchemy import create_engine
from config import config

try:
    params = config()
    engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**params))

    # Save the dataframe to PostgreSQL (make sure to type table names in lower cases)
    df_TaxonomyForSQL.to_sql('taxonomyschema_generaldts', con=engine, if_exists='replace', index=False)
    #df_RoleTypeForSQL.to_sql('roletype_generaldts', con=engine, if_exists='replace', index=False)
except:
    print ('Error:', sys.exc_info()[0], sys.exc_info()[1])

Error: <class 'sqlalchemy.exc.ProgrammingError'> (psycopg2.ProgrammingError) can't adapt type 'dict'
[SQL: INSERT INTO roletype_generaldts (tag, id, type, role, arcrole, base, originalpath, fragment_id, href_type, searchablepath, "roleURI", "arcroleURI", "cyclesAllowed", "childElementDict") VALUES (%(tag)s, %(id)s, %(type)s, %(role)s, %(arcrole)s, %(base)s, %(originalpath)s, %(fragment_id)s, %(href_type)s, %(searchablepath)s, %(roleURI)s, %(arcroleURI)s, %(cyclesAllowed)s, %(childElementDict)s)]
[parameters: ({'tag': 'roleType', 'id': 'rol_std_CabinetOfficeOrdinanceOnDisclosureOfCorporateInformationEtcFormNo2SecuritiesRegistrationStatement', 'type': None, 'role': None, 'arcrole': None, 'base': None, 'originalpath': None, 'fragment_id': None, 'href_type': 'No href', 'searchablepath': None, 'roleURI': 'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_std_CabinetOfficeOrdinanceOnDisclosureOfCorporateInformationEtcFormNo2SecuritiesRegistrationStatement', 'arcroleURI': None, 'cyclesAllowed